In [1]:
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cu116.html
!pip install torch-geometric
!pip install ogb

from IPython.display import clear_output
clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')
from datetime import datetime, timedelta, date
import torch
import random
import os
import math

from torch_geometric.data import Data
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:31: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_scatter/_scatter_cuda.so: undefined symbol: _ZN2at4_ops6narrow4callERKNS_6TensorElll
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.10/dist-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.10/dist-packages/torch_sparse/_diag_cuda.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [4]:
raw_dataset_path = '/content/drive/MyDrive/Summer_Invitational_2023_Datathon_Datasets/Raw-Datasets'
processed_dataset_path = '/content/drive/MyDrive/Summer_Invitational_2023_Datathon_Datasets/Processed-Datasets'

In [5]:
def convert_to_Data(graph_path):
  x_path = os.path.join(graph_path, 'x.pt')
  y_path = os.path.join(graph_path, 'y.pt')
  edge_index_path = os.path.join(graph_path, 'edge_list.pt')
  edge_attr_path = os.path.join(graph_path, 'edge_attr.pt')

  x = torch.load(x_path).to(torch.float)
  y = torch.load(y_path).to(torch.float)
  edge_index = torch.load(edge_index_path)
  edge_attr = torch.load(edge_attr_path).to(torch.float)

  return Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr)

In [6]:
class GraphDataset(Dataset):
  def __init__(self, root, transform=None):

    self.data = []
    for graph_folder in tqdm(os.listdir(root)):
      graph_path = os.path.join(root, graph_folder)
      self.data.append(convert_to_Data(graph_path))

    self.create_idx_split()

    self.task_type = "regression"

    self.eval_metric = "rmse"


  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      # If the input 'idx' is a tensor, return a list of data items corresponding to the indices
      return [self.data[i] for i in idx]
    else:
      return self.data[idx]

  def create_idx_split(self):
    split = {}

    avail = list(range(len(self.data)))

    train_prop = 0.7
    val_prop = 0.15
    test_prop = 0.15

    num_train = int(train_prop * len(self.data))
    num_val = int(val_prop * len(self.data))
    num_test = len(self.data) - num_train - num_val

    train_split = random.sample(avail, num_train)
    avail = list(set(avail) - set(train_split))

    val_split = random.sample(avail, num_val)
    avail = list(set(avail) - set(val_split))

    test_split = random.sample(avail, num_test)
    avail = list(set(avail) - set(test_split))

    split['train'] = torch.tensor(train_split)
    split['valid'] = torch.tensor(val_split)
    split['test'] = torch.tensor(test_split)

    self.split = split

  def get_idx_split(self):
    return self.split


# RUN THIS CELL BELOW TO CONVERT RAW DATASET TO PROCESSED DATASET

In [12]:
ds = 0
for airline_folder in os.listdir(raw_dataset_path):
  if airline_folder != "WN":
    continue
  print("Saving " + airline_folder + " dataset.")
  airline_path = os.path.join(raw_dataset_path, airline_folder)

  ds = GraphDataset(airline_path)

  torch.save(ds, os.path.join(processed_dataset_path, airline_folder + '.pt'))

Saving WN dataset.


100%|██████████| 365/365 [04:24<00:00,  1.38it/s]


In [ ]:
for d in ds:
  print(d.x)

TypeError: ignored

In [ ]:
airline_path

'/content/drive/MyDrive/Summer_Invitational_2023_Datathon_Datasets/Raw-Datasets/HA'